# MultiScale RoIAlign

In [1]:
from flame.core.model.maskRCNN.faster_rcnn import fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
from flame.core.model.maskRCNN.faster_rcnn import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from flame.core.model.maskRCNN.mask_rcnn import maskrcnn_resnet50_fpn, MaskRCNN

In [2]:
model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block):

In [3]:
import cv2
import torch
import numpy as np

image = cv2.imread('/home/phungpx/Downloads/dog.jpg')
sample = torch.from_numpy(image)
sample = sample.float().div(255.)
sample = sample.permute(2, 0, 1)

In [4]:
preds = model([sample])

In [5]:
preds

[{'boxes': tensor([[ 38.6951,  75.0016, 548.5908, 505.1139]], grad_fn=<StackBackward0>),
  'labels': tensor([18]),
  'scores': tensor([0.9993], grad_fn=<IndexBackward0>)}]